In [137]:
from ipynb.fs.full.data_extraction import read_file,init_spark,create_leagues
from pyspark.sql.functions import lit,sum,format_number,log
from matplotlib import pyplot as plt
%matplotlib qt

In [50]:
spark = init_spark()

In [51]:
match = read_file("Match.csv")
leagues = create_leagues()

In [52]:
book_keeper = ["B365H","B365D","B365A"]
match = match.select(["league_id","season"] + book_keeper)
match = match.na.drop()

In [53]:
match_with_probs = match
for keeper in book_keeper:
    match_with_probs = match_with_probs.withColumn(keeper, 1/match_with_probs[keeper])

In [54]:
match_with_probs = match_with_probs.withColumn("n_B365H", format_number(match_with_probs["B365H"]/(match_with_probs["B365H"]+match_with_probs["B365D"]
                                                                 +match_with_probs["B365A"]),2))
match_with_probs = match_with_probs.withColumn("n_B365D", format_number(match_with_probs["B365D"]/(match_with_probs["B365H"]+match_with_probs["B365D"]
                                                                 +match_with_probs["B365A"]),2))
match_with_probs = match_with_probs.withColumn("n_B365A", format_number(match_with_probs["B365A"]/(match_with_probs["B365H"]+match_with_probs["B365D"]
                                                                 +match_with_probs["B365A"]),2))


In [92]:
match_entropy = match_with_probs.withColumn("entropy",format_number(-(match_with_probs["n_B365H"]*log(3.0,match_with_probs["n_B365H"])
                                       +match_with_probs["n_B365D"]*log(3.0,match_with_probs["n_B365D"])
                                       +match_with_probs["n_B365A"]*log(3.0,match_with_probs["n_B365A"])),2).cast("double"))

In [93]:
match_entropy = match_entropy.groupBy("season","league_id").mean()\
.select("season","league_id",format_number("avg(entropy)",2).alias("avg_entropy"))

In [97]:
final_pd = match_entropy.join(leagues, leagues["id"] == match_entropy["league_id"]).drop("id","league_name","league_id")\
.withColumnRenamed("name","country").orderBy("season").toPandas()

In [131]:
leagues_pd = leagues.toPandas()
entropy_mean = final_pd.reset_index().pivot(index='season', columns='country', values='avg_entropy').drop("Belgium",axis=1)
for col in entropy_mean.columns.values:
    entropy_mean[col]=entropy_mean[col].astype("str").astype("float")

In [138]:
entropy_mean.plot(figsize=(12,10),marker='o')
plt.show()